# Import library and Data

In [1]:
import tensorflow as tf
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Lasso 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split

from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score 
#scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
#                         scoring="neg_mean_squared_error", cv=10)

#rmse_scores = np.sqrt(-scores)

import tensorflow as tf
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Lasso 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split

from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import LinearSVC

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score 
#scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
#                         scoring="neg_mean_squared_error", cv=10)

#rmse_scores = np.sqrt(-scores)



from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC


from sklearn.metrics import mean_squared_error

from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC


from sklearn.metrics import mean_squared_error

from sklearn.metrics import accuracy_score

In [2]:
file_name = "BRI641_fMRI_data_new_sets.mat"
rawdata = loadmat(file_name)

len(rawdata)

7

The mat data file is loaded using the function loadmat()

# Check the Data

and then we need to check the data structure.


In [15]:
print(rawdata)


{'data_1d_x': array([[-0.03519887, -0.19054532, -0.7751938 , ...,  0.82167444,
        -0.15327348,  0.70570284],
       [-0.50451719, -0.55348879, -0.7751938 , ..., -0.06662225,
        -0.15327348,  0.32424185],
       [-0.03519887, -0.19054532, -0.36175711, ..., -0.06662225,
        -0.15327348, -0.05721915],
       ..., 
       [-0.19350239, -0.34979951, -0.46656299, ..., -0.02357934,
        -0.86529217, -1.40392241],
       [-0.19350239,  0.33273611,  0.26530052, ...,  0.29081192,
        -0.86529217, -0.54656521],
       [-1.00824931, -1.03233512, -0.83249474, ...,  0.29081192,
        -0.86529217, -1.40392241]]), 'data_y': array([[0],
       [0],
       [0],
       ..., 
       [3],
       [3],
       [3]], dtype=uint8), 'vMsk': array([[   340],
       [   392],
       [   393],
       ..., 
       [151974],
       [151983],
       [152035]]), '__version__': '1.0', 'data_3d_x': array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         

This data consists of three main parts.

1. data_1d_x : 1440 * 74484

2. data_3d_x & Vmsk  : 53 * 63 * 46 * 1440

3. y : Dependent Variable : 1440 * 1

In [16]:
#print(rawdata["data_3d_x"])
print(rawdata["data_3d_x"].shape)

(53, 63, 46, 1440)


In [17]:
print(rawdata["data_1d_x"])
print(rawdata["data_1d_x"].shape)

[[-0.03519887 -0.19054532 -0.7751938  ...,  0.82167444 -0.15327348
   0.70570284]
 [-0.50451719 -0.55348879 -0.7751938  ..., -0.06662225 -0.15327348
   0.32424185]
 [-0.03519887 -0.19054532 -0.36175711 ..., -0.06662225 -0.15327348
  -0.05721915]
 ..., 
 [-0.19350239 -0.34979951 -0.46656299 ..., -0.02357934 -0.86529217
  -1.40392241]
 [-0.19350239  0.33273611  0.26530052 ...,  0.29081192 -0.86529217
  -0.54656521]
 [-1.00824931 -1.03233512 -0.83249474 ...,  0.29081192 -0.86529217
  -1.40392241]]
(1440, 74484)


In [18]:
print(rawdata["data_y"])
print(rawdata["data_y"].shape)

[[0]
 [0]
 [0]
 ..., 
 [3]
 [3]
 [3]]
(1440, 1)


In [19]:
print(rawdata["vMsk"])
print(rawdata["vMsk"].shape)

[[   340]
 [   392]
 [   393]
 ..., 
 [151974]
 [151983]
 [152035]]
(74484, 1)


I used data_1d_x data.

In order to process LOOCV(Leave One Out Cross-Validation), 

we divided into 12 part(participants)used LeaveOneGruopOut() function.



In [3]:
X_data=rawdata["data_1d_x"]
y_data = (rawdata["data_y"])


groups = []
for i in range(1,13):
    group = [i] * 120
    for i in group:
        groups.append(i)
LOOCV = LeaveOneGroupOut()
LOOCV.get_n_splits(X_data,y_data, groups)


12

# Setup

In [4]:
def reset_graph():  # initialize graph
    tf.reset_default_graph()



First , I applied SVM.

1. SVM Polynomial Kernel


# Polynomial kernel SVM

I have considered two combinations of hyper Parameters.

1. Degree =3 , coef0 =1 and C=5 : 
Degree means degree of polynomial kernel, so 3 - degree polynomial kernel in this option 

Coef0 controls how much the model is influenced by high degree polynomials vs low degree polynomials.

smaller C leads to a wider street but more margin violations.  


2. Degree =10 , coef0 =100 and C=5 : 

Degree means degree of polynomial kernel, so 10 - degree polynomial kernel in this option.

I used the same values C as before.

In [10]:
clf = SVC()

max_test = []
for train_index, test_index in LOOCV.split(X_data, y_data, groups):
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    poly_kernel_svm_clf = Pipeline((
            ("scaler", StandardScaler()),
            ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
        ))
    poly_kernel_svm_clf.fit(X_train, y_train)
    poly_y_pred = poly_kernel_svm_clf.predict(X_train)
    poly_acc_train = accuracy_score(y_train, poly_y_pred)
    poly_y_pred = poly_kernel_svm_clf.predict(X_test)
    poly_acc_test = accuracy_score(y_test, poly_y_pred)
    
    max_test.append(poly_acc_test)
    print("Train accuracy:", poly_acc_train, "Test accuracy:", poly_acc_test)
    
print(max_test) 
print("Mean Test accuracy: ", np.mean(max_test))

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy: 1.0 Test accuracy: 0.908333333333


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy: 1.0 Test accuracy: 0.983333333333


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Train accuracy: 1.0 Test accuracy: 0.883333333333


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy: 1.0 Test accuracy: 0.983333333333


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy: 1.0 Test accuracy: 0.933333333333


In the first combination of hyperparameters in polynomial kernel, the accuracy of each LOOCV is as follows.

In [11]:
    
print(max_test) 
print("Mean Test accuracy: ", np.mean(max_test))

[0.90833333333333333, 0.98333333333333328, 0.98333333333333328, 0.84999999999999998, 0.90833333333333333, 0.95833333333333337, 0.9916666666666667, 0.97499999999999998, 0.94166666666666665, 0.8833333333333333, 0.98333333333333328, 0.93333333333333335]
Mean Test accuracy:  0.941666666667


The first polynomial kernel SVM average accuracy of 12 parts is  0.9417

In the Second combination of hyperparameters in polynomial kernel,

the accuracy of each LOOCV is as follows.

In [14]:
clf = SVC()

max_test = []
for train_index, test_index in LOOCV.split(X_data, y_data, groups):
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    poly100_kernel_svm_clf = Pipeline((
            ("scaler", StandardScaler()),
            ("svm_clf", SVC(kernel="poly", degree=10, coef0=100, C=5))
        ))
    poly100_kernel_svm_clf.fit(X_train, y_train)
    poly100_y_pred = poly100_kernel_svm_clf.predict(X_train)
    poly100_acc_train = accuracy_score(y_train, poly100_y_pred)
    poly100_y_pred = poly100_kernel_svm_clf.predict(X_test)
    poly100_acc_test = accuracy_score(y_test, poly100_y_pred)
    
    max_test.append(poly100_acc_test)
    print("Train accuracy:", poly100_acc_train, "Test accuracy:", poly100_acc_test)
    
print(max_test) 
print("Mean Test accuracy: ", np.mean(max_test))

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy: 1.0 Test accuracy: 0.9


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy: 1.0 Test accuracy: 0.966666666667


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

Train accuracy: 1.0 Test accuracy: 0.9


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy: 1.0 Test accuracy: 0.983333333333


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train accuracy: 1.0 Test accuracy: 0.95
[0.90000000000000002, 0.96666666666666667, 1.0, 0.89166666666666672, 0.91666666666666663, 0.95833333333333337, 0.9916666666666667, 0.9916666666666667, 0.94166666666666665, 0.90000000000000002, 0.98333333333333328, 0.94999999999999996]
Mean Test accuracy:  0.949305555556


The second polynomial kernel SVM model average accuracy of 12 parts is 0.9493


In [15]:
    
print(max_test) 
print("Mean Test accuracy: ", np.mean(max_test))

[0.90000000000000002, 0.96666666666666667, 1.0, 0.89166666666666672, 0.91666666666666663, 0.95833333333333337, 0.9916666666666667, 0.9916666666666667, 0.94166666666666665, 0.90000000000000002, 0.98333333333333328, 0.94999999999999996]
Mean Test accuracy:  0.949305555556


So, Second polynomial kernel SVM model is better than first polynomial kernel SVM model




# DNN


I wanted to create and analyze deep neural networks(artificial neural networks)

First of all, i made up two hidden layers.

The first hidden layer consists of 300 neurons.

and the second hidden layer consists of 100 neurons.


# Using dense() instead of neuron_layer()

If initializer is None (the default), the default initializer passed in the variable scope will be used. If that one is None too, a glorot_uniform_initializer will be used. The initializer can also be a Tensor, in which case the variable is initialized to this value and shape

In [19]:
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 4

reset_graph()

X = tf.placeholder(tf.float32, shape=(None,74484), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

# print(y)

with tf.name_scope("dnn"): # Set hidden layers
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
    
with tf.name_scope("loss"): # Set loss function : cross-entropy
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"): # Set the Learning rate : 0.01
    optimizer = tf.train.GradientDescentOptimizer(learning_rate) # Optimizer : GradientDescent
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1) # Set the highest values as predict class 
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()



In [28]:
n_epochs = 10  # set the number of epochs to 10
batch_size = 10  # set the number of mini-batch size to 10


total = []  # set the space to store the accuracy of the entire LOOCV, later
for train_index, test_index in LOOCV.split(X_data, y_data, groups): # LOOCV
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    with tf.Session() as sess:
        init.run()
        max_test = [] # set the space to store all the accuracy of model with the mini-batch , later
        for epoch in range(n_epochs):
            for iteration in range(0,len(y_train) // batch_size):
                X_batch = X_train[(batch_size*iteration):batch_size*(iteration+1)]
                # Load as many as mini-batch size  
                
                # print(X_batch.shape) # Check the mini-batch
                y_batch = y_train[(batch_size*iteration):batch_size*(iteration+1)].reshape(-1,)
                
                
                # print(a.shape)
                # y_batch= a.reshape(-1,)
                # print(y_batch.shape)
                
                # run the DNN
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch.reshape(-1,)})
            #test12 = correct.eval(feed_dict={X: X_batch, y: y_batch})
            acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train.reshape(-1,)})
            #test23 = correct.eval(feed_dict={X: X_test, y: y_test.reshape(-1,)})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape(-1,)})
            # show the accuracy of the test and train set
            
            
            max_test.append(acc_test)
            #print(test12)
            #print("------------------------")
            #print(test23)
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        total.append(max_test[n_epochs-1])
        
        save_path = saver.save(sess, "./my_model_final.ckpt")
        print("End_LOOCV")
        print(max_test[n_epochs-1]) # Show the accuracy at the last epoch
        print("--"*30) 
        
        
print("=="*30)
print("final result")
print(total)
print("Test accuracy: ", np.mean(total)) # Show average accuracy at each LOOCV



0 Train accuracy: 0.741667 Test accuracy: 0.641667
1 Train accuracy: 0.857576 Test accuracy: 0.691667
2 Train accuracy: 0.889394 Test accuracy: 0.775
3 Train accuracy: 0.952273 Test accuracy: 0.808333
4 Train accuracy: 0.959848 Test accuracy: 0.816667
5 Train accuracy: 0.968939 Test accuracy: 0.783333
6 Train accuracy: 0.987121 Test accuracy: 0.875
7 Train accuracy: 0.982576 Test accuracy: 0.85
8 Train accuracy: 0.996212 Test accuracy: 0.841667
9 Train accuracy: 0.999242 Test accuracy: 0.866667
End_LOOCV
0.866667
------------------------------------------------------------
0 Train accuracy: 0.638636 Test accuracy: 0.625
1 Train accuracy: 0.831818 Test accuracy: 0.725
2 Train accuracy: 0.906061 Test accuracy: 0.858333
3 Train accuracy: 0.941667 Test accuracy: 0.941667
4 Train accuracy: 0.959091 Test accuracy: 0.933333
5 Train accuracy: 0.975 Test accuracy: 0.966667
6 Train accuracy: 0.986364 Test accuracy: 0.991667
7 Train accuracy: 0.993182 Test accuracy: 0.991667
8 Train accuracy: 0.9

As a result, the average accurcy of DNN(2 hidden layers) is 0.9264

Next, i checked other DNN with three hidden layers.

The first hidden layer consists of 100 neurons, second hidden layer consists of 200 neurons.

and the third hidden layer consists of 100 neurons.


In [29]:
n_hidden1 = 100
n_hidden2 = 200
n_hidden3 = 100

n_outputs = 4

reset_graph()

X = tf.placeholder(tf.float32, shape=(None,74484), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

# print(y)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()



Other settings were the same.

In [30]:
n_epochs = 10
batch_size = 10


total = []
for train_index, test_index in LOOCV.split(X_data, y_data, groups):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    with tf.Session() as sess:
        init.run()
        max_test = []
        for epoch in range(n_epochs):
            for iteration in range(0,len(y_train) // batch_size):
    #             print(X_)
                X_batch = X_train[(batch_size*iteration):batch_size*(iteration+1)]
    #             print(X_batch.shape)
                y_batch = y_train[(batch_size*iteration):batch_size*(iteration+1)].reshape(-1,)
    #             print(a.shape)
    #             y_batch= a#.reshape(-1,)
    #             print(y_batch.shape)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch.reshape(-1,)})
            #test12 = correct.eval(feed_dict={X: X_batch, y: y_batch})
            acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train.reshape(-1,)})
            #test23 = correct.eval(feed_dict={X: X_test, y: y_test.reshape(-1,)})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape(-1,)})
            max_test.append(acc_test)
            #print(test12)
            #print("------------------------")
            #print(test23)
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        total.append(max_test[n_epochs-1])
        
        save_path = saver.save(sess, "./my_model_final.ckpt")
        print("End_LOOCV")
        print(max_test[n_epochs-1])
        print("--"*30) 
        
        

print("=="*30)
print("final result")
print(total)
print("Test accuracy: ", np.mean(total))

0 Train accuracy: 0.835606 Test accuracy: 0.725
1 Train accuracy: 0.830303 Test accuracy: 0.666667
2 Train accuracy: 0.946212 Test accuracy: 0.825
3 Train accuracy: 0.967424 Test accuracy: 0.85
4 Train accuracy: 0.985606 Test accuracy: 0.908333
5 Train accuracy: 0.986364 Test accuracy: 0.816667
6 Train accuracy: 0.993939 Test accuracy: 0.891667
7 Train accuracy: 1.0 Test accuracy: 0.925
8 Train accuracy: 1.0 Test accuracy: 0.925
9 Train accuracy: 1.0 Test accuracy: 0.925
End_LOOCV
0.925
------------------------------------------------------------
0 Train accuracy: 0.634848 Test accuracy: 0.541667
1 Train accuracy: 0.888636 Test accuracy: 0.933333
2 Train accuracy: 0.962879 Test accuracy: 0.983333
3 Train accuracy: 0.899242 Test accuracy: 0.85
4 Train accuracy: 0.980303 Test accuracy: 0.966667
5 Train accuracy: 0.664394 Test accuracy: 0.508333
6 Train accuracy: 0.439394 Test accuracy: 0.466667
7 Train accuracy: 0.523485 Test accuracy: 0.5
8 Train accuracy: 0.682576 Test accuracy: 0.525


The accuracy of the last epoch was selected, and the accuracy of each part was as follows.

[0.92500001, 0.73333335, 1.0, 0.89999998, 0.85000002, 0.92500001, 0.93333334, 0.66666669, 0.80000001, 0.89166665, 0.81666666, 0.86666667]

The total average accuracy of this model is 0.8590.


As a result, it can be seen that the accuracy is not always impoved as the number of layers deeper.


# Convolutional Neural Net

Since i used on-dimesional data, we had to use the appropriate tensorflow function( tf.layers.conv1d, tf.layers.max_pooling1d).

I wanted to make a CNN similar to AlexNet, so the architectures of CNN is as follows.

1 layer : CONVOLUTION LAYER 32 feature maps, kernel size : 10, Strides : 5, Zero-padding : Done
, neurons : ReLU

2 layer : POOLING LAYER 32 feature maps, kernel size : 2, Strides : 2, Zero-padding : Not

3 layer : CONVOLUTION LAYER 64 feature maps, kernel size : 10, Strides : 5, Zero-padding : Done
, neurons : ReLU

4 layer : POOLING LAYER 64 feature maps, kernel size : 2, Strides : 2, Zero-padding : Not
At this time, 50% of the neurons dropout

5 layer : NEURAL LAYER, neurons : ReLU
At this time, 75% of the neurons dropout


Optimizer : ADAM

Batch Size : 30

Epoch Size : 10 

In [7]:

width = 74484
channels = 1
n_inputs = 1*74484


conv1_fmaps = 32
conv1_ksize = 10
conv1_stride = 5
conv1_pad = "SAME"


pool2_fmaps = conv1_fmaps

conv3_fmaps = 64
conv3_ksize = 10
conv3_stride = 5
conv3_pad = "SAME"
conv3_dropout_rate = 0.5
fc1_dropout_rate = 0.25
pool4_fmaps = conv3_fmaps

n_fc1 = 64
n_outputs = 4

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=(None,74484), name="X")
    X_reshaped = tf.reshape(X, shape=[-1, width, channels])
    y = tf.placeholder(tf.int64, shape=(None), name="y")
    training = tf.placeholder_with_default(False, shape=[], name='training')





with tf.name_scope("conv1"):
    conv1 = tf.layers.conv1d(X_reshaped,
                         filters=conv1_fmaps,
                         kernel_size=conv1_ksize,
                         strides=conv1_stride,
                         padding=conv1_pad,
                         activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling1d(conv1,
                                    pool_size=2,
                                    strides=2,
                                    padding="VALID")


with tf.name_scope("conv3"):
    conv3 = tf.layers.conv1d(pool2,
                             filters=conv3_fmaps,
                             kernel_size=conv3_ksize,
                             strides=conv3_stride,
                             padding=conv3_pad,
                             activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling1d(conv3,
                                    pool_size=2,
                                    strides=2,
                                    padding="VALID")
    pool4_flat = tf.reshape(pool4, shape=[-1, pool4_fmaps*745])
    pool4_flat_drop = tf.layers.dropout(pool4_flat,
                                        conv3_dropout_rate,
                                        training=training)
    
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool4_flat_drop, n_fc1, activation=tf.nn.relu, name="fc1")
    fc1_drop = tf.layers.dropout(fc1, fc1_dropout_rate, training=training)

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1_drop, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


In [9]:
n_epochs = 15
batch_size = 30

total = []
for train_index, test_index in LOOCV.split(X_data, y_data, groups):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

    with tf.Session() as sess:
        init.run()
        max_test = []
        for epoch in range(n_epochs):
            for iteration in range(0,len(y_train) // batch_size):
    #             print(X_)
                X_batch = X_train[(batch_size*iteration):batch_size*(iteration+1)]
    #             print(X_batch.shape)
                y_batch = y_train[(batch_size*iteration):batch_size*(iteration+1)].reshape(-1,)
    #             print(a.shape)
    #             y_batch= a#.reshape(-1,)
    #             print(y_batch.shape)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch.reshape(-1,)})
            acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train.reshape(-1)})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape(-1)})
            max_test.append(acc_test)
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

            
        total.append(max_test[n_epochs-1])
        
        save_path = saver.save(sess, "./my_model_final.ckpt")
        print("End_LOOCV")
        print(max_test[n_epochs-1])
        print("--"*30) 
        
print("=="*30)
print("final result")
print(total)
print("Test accuracy: ", np.mean(total))

0 Train accuracy: 0.630303 Test accuracy: 0.583333
1 Train accuracy: 0.765909 Test accuracy: 0.691667
2 Train accuracy: 0.952273 Test accuracy: 0.891667
3 Train accuracy: 0.943182 Test accuracy: 0.908333
4 Train accuracy: 0.968182 Test accuracy: 0.891667
5 Train accuracy: 0.984091 Test accuracy: 0.966667
6 Train accuracy: 0.994697 Test accuracy: 0.966667
7 Train accuracy: 0.993939 Test accuracy: 0.958333
8 Train accuracy: 0.992424 Test accuracy: 0.925
9 Train accuracy: 0.990152 Test accuracy: 0.958333
10 Train accuracy: 0.99697 Test accuracy: 0.975
11 Train accuracy: 0.998485 Test accuracy: 0.966667
12 Train accuracy: 1.0 Test accuracy: 0.975
13 Train accuracy: 1.0 Test accuracy: 0.95
14 Train accuracy: 1.0 Test accuracy: 0.966667
End_LOOCV
0.966667
------------------------------------------------------------
0 Train accuracy: 0.440909 Test accuracy: 0.458333
1 Train accuracy: 0.490152 Test accuracy: 0.558333
2 Train accuracy: 0.790152 Test accuracy: 0.758333
3 Train accuracy: 0.943182

1 Train accuracy: 0.556061 Test accuracy: 0.383333
2 Train accuracy: 0.943182 Test accuracy: 0.75
3 Train accuracy: 0.984091 Test accuracy: 0.941667
4 Train accuracy: 0.981061 Test accuracy: 0.991667
5 Train accuracy: 0.99697 Test accuracy: 0.966667
6 Train accuracy: 0.993939 Test accuracy: 0.958333
7 Train accuracy: 0.987879 Test accuracy: 0.925
8 Train accuracy: 0.967424 Test accuracy: 0.783333
9 Train accuracy: 0.998485 Test accuracy: 0.941667
10 Train accuracy: 0.999242 Test accuracy: 0.916667
11 Train accuracy: 1.0 Test accuracy: 0.925
12 Train accuracy: 1.0 Test accuracy: 0.958333
13 Train accuracy: 1.0 Test accuracy: 0.958333
14 Train accuracy: 1.0 Test accuracy: 0.958333
End_LOOCV
0.958333
------------------------------------------------------------
0 Train accuracy: 0.579545 Test accuracy: 0.525
1 Train accuracy: 0.915152 Test accuracy: 0.875
2 Train accuracy: 0.809091 Test accuracy: 0.775
3 Train accuracy: 0.925758 Test accuracy: 0.891667
4 Train accuracy: 0.97197 Test accura

The accuracy of CNN's last epoch was selected, and the accuracy of each part was as follows.

[0.96666664, 0.99166667, 1.0, 0.95833331, 0.89166665, 0.99166667, 1.0, 1.0, 0.94999999, 0.95833331, 0.95833331, 0.97500002]


The total average accuracy of this model is  0.9701.



# Result

The total average accuracy of CNN is 0.9701, and

the average accurcy of DNN(2 hidden layers) is 0.9264 .

The total average accuracy of polynomial SVM is 0.9493 .


So, I chose CNN as the final model, because the higher accuracy.

but more computational time.

If you had more epoch sizes, you can improve performance a little more. but you will not be able to guarantee a higher accuracy considering the speed of computational time.
